In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow-gpu

In [ ]:
import os
import cv2
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle
import seaborn as sn; sn.set(font_scale=1.4)
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D


In [ ]:
# Define the folders containing the videos
extrovert_folder = '/content/drive/MyDrive/PFA/Videos/train/Extrovert'
introvert_folder = '/content/drive/MyDrive/PFA/Videos/train/Introvert'

In [ ]:
# Load the videos and extract the frames
frames = []
labels = []
for folder, label in [(extrovert_folder, 0), (introvert_folder, 1)]:
    for filename in os.listdir(folder):
        if filename.endswith('.mp4'):
            video_path = os.path.join(folder, filename)
            cap = cv2.VideoCapture(video_path)
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (224, 224))
                frames.append(frame)
                labels.append(label)
            cap.release()
frames

In [ ]:
# Convert the frames to a numpy array and preprocess them
frames = np.array(frames, dtype=np.float32) / 255.0
frames -= np.array([0.485, 0.456, 0.406])
frames /= np.array([0.229, 0.224, 0.225])

In [ ]:
# Define a simple CNN model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Split the data into training and validation sets
num_samples = len(frames)
split_idx = int(0.8 * num_samples)
train_frames, train_labels = frames[:split_idx], labels[:split_idx]
val_frames, val_labels = frames[split_idx:], labels[split_idx:]
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [ ]:
# Train the model
history = model.fit(train_frames, train_labels, epochs=5, batch_size=32, validation_data=(val_frames, val_labels))

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(val_frames, val_labels)
print('Test accuracy:', test_acc)